# 1. Import Dependencies

In [ ]:
!pip install opencv-python

In [1]:
# Import opencv
import cv2
# Import Operating System
import os

# 2. Define Images to Collect

In [ ]:
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
number_imgs = 5